# **[ Kaggle 프로젝트 ] 딥 러닝 기반 회귀 분석**

## **Google Drive Mount**

In [72]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


•USA Real Estate Dataset

▪ 주어진 csv 파일을 읽어온 후 회귀 Task를 진행하세요.

1. 주어진 csv 파일 내에 존재하는 데이터가 이상치를 포함하고 있는지 확인하고, 만약 확인이 되면 이를 제거하세요.
2. “price” 피처 Series를 y변수에 저장하고, 나머지 피처들을 x변수에 저장하세요.
3. x의 피처들에 상황에 맞는 적절한 전처리를 적용하신 후, 결과 값들을 다시 x에 저장하세요.
4. y의 데이터에 상황에 맞는 적절한 전처리를 적용하신 후, 결과 값들을 다시 y에 저장하세요.
5. x와 y를 훈련, 검증, 그리고 시험 데이터 셋으로 나누세요. (Scikit-Learn의 train_test_split 사용 → Seed는 42로 고정)

- test_size → 0.2
- random_state → 42

5. Seed를 42로 고정한 후 상황에 맞는 적절한 딥 러닝 회귀 모델을 만든 후 MSE Loss를 사용하여 훈련을 진행하세요.
6. 훈련 과정에서 얻은 Train Loss와 Validation Loss를 시각화하세요.
7. 훈련된 모델과 시험 데이터 셋을 사용하여 모든 정량적 지표를 (MSE, MAE, RMSE) 출력하세요.
8. 앞선 모든 과정에서 모델의 구조, 하이퍼파라미터, 손실 함수 등을 바꾸어 가면서 모델의 성능을 높여보세요.

In [73]:
import pandas as pd

In [74]:
file_path = '/content/drive/MyDrive/KOSA/realtor-data.zip.csv'

In [75]:
df = pd.read_csv(file_path)

In [76]:
df.head(5)

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


이상치 제거

In [77]:
import numpy as np

# 1) 수치형 컬럼만 선택
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"수치형 컬럼: {numeric_cols}")

# 2) 각 컬럼의 Q1, Q3, IQR 계산
Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# 3) 이상치가 아닌 조건
condition = ~((df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# 4) 이상치 제거한 새로운 DataFrame 만들기
df_clean = df[condition]

print(f"원본 행 수: {len(df)}")
print(f"이상치 제거 후 행 수: {len(df_clean)}")


수치형 컬럼: ['brokered_by', 'price', 'bed', 'bath', 'acre_lot', 'street', 'zip_code', 'house_size']
원본 행 수: 2226382
이상치 제거 후 행 수: 1676707


In [78]:
df = df_clean.copy()

In [79]:
y=df['price']
x=df.drop('price', axis=1)

In [80]:
y

,price
0,105000.0
1,80000.0
2,67000.0
3,145000.0
5,179000.0
...,...
2226376,339900.0
2226377,359900.0
2226378,350000.0
2226380,179900.0


In [81]:
from sklearn.preprocessing import LabelEncoder

x의 피처들에 상황에 맞는 적절한 전처리를 적용하신 후, 결과 값들을 다시 x에 저장하세요.(label encoding)

In [82]:
cat_cols = x.select_dtypes(include=['object']).columns.tolist()
print(f"Label Encoding 대상 컬럼: {cat_cols}")

le = LabelEncoder()

for col in cat_cols:
  x[col] = le.fit_transform(x[col])

Label Encoding 대상 컬럼: ['status', 'city', 'state', 'prev_sold_date']


In [83]:
print(x.head())

   brokered_by  status  bed  bath  acre_lot     street   city  state  \
0     103378.0       0  3.0   2.0      0.12  1962661.0     77     41   
1      52707.0       0  4.0   2.0      0.08  1902874.0     77     41   
2     103379.0       0  2.0   1.0      0.15  1404990.0   7672     41   
3      31239.0       0  4.0   2.0      0.10  1947675.0  12473     41   
5     103378.0       0  4.0   3.0      0.46  1850806.0  13847     41   

   zip_code  house_size  prev_sold_date  
0     601.0       920.0           14168  
1     601.0      1527.0           14168  
2     795.0       748.0           14168  
3     731.0      1800.0           14168  
5     612.0      2520.0           14168  


y의 데이터에 상황에 맞는 적절한 전처리를 적용하신 후, 결과 값들을 다시 y에 저장하세요.

In [84]:
import numpy as np

In [85]:
y = np.log1p(y)
print(y.head())

0    11.561725
1    11.289794
2    11.112463
3    11.884496
5    12.095147
Name: price, dtype: float64


In [86]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
y = scaler.fit_transform(y.values.reshape(-1,1)).flatten()

print(y[:5])


[0.82966695 0.81015325 0.79742798 0.85282891 0.86794516]


In [87]:
print(y.min())

nan


In [98]:
print(df['price'].min())
print(df['price'].max())


0.0
1127350.0


x와 y를 훈련, 검증, 그리고 시험 데이터 셋으로 나누세요. (Scikit-Learn의 train_test_split 사용 → Seed는 42로 고정)
test_size → 0.2
random_state → 42

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
x_temp, x_test, y_temp, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [90]:
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)

In [91]:
print(f"Train shape: {x_train.shape}, {y_train.shape}")
print(f"Validation shape: {x_val.shape}, {y_val.shape}")
print(f"Test shape: {x_test.shape}, {y_test.shape}")

Train shape: (1073092, 11), (1073092,)
Validation shape: (268273, 11), (268273,)
Test shape: (335342, 11), (335342,)


Seed를 42로 고정한 후 상황에 맞는 적절한 딥 러닝 회귀 모델을 만든 후 MSE Loss를 사용하여 훈련을 진행하세요.

In [92]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

torch.manual_seed(42)
np.random.seed(42)

In [93]:
# 넘파이 → 텐서로 변환
x_train_tensor = torch.tensor(np.array(x_train), dtype=torch.float32)
y_train_tensor = torch.tensor(np.array(y_train), dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(np.array(x_val), dtype=torch.float32)
y_val_tensor = torch.tensor(np.array(y_val), dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(np.array(x_test), dtype=torch.float32)
y_test_tensor = torch.tensor(np.array(y_test), dtype=torch.float32).view(-1, 1)


In [94]:
input_dim = x_train.shape[1]

In [95]:
model = nn.Sequential(
    nn.Linear(input_dim, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)
print(model)

Sequential(
  (0): Linear(in_features=11, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
)


In [96]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [97]:
epochs = 200

for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()

  output = model(x_train_tensor)
  loss = criterion(output, y_train_tensor)
  loss.backward()
  optimizer.step()

  if(epoch + 1 ) % 20 == 0:
    model.eval()
    val_output = model(x_val_tensor)
    val_loss = criterion(val_output, y_val_tensor)
    print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

Epoch [20/200] - Train Loss: nan | Val Loss: nan
Epoch [40/200] - Train Loss: nan | Val Loss: nan


KeyboardInterrupt: 